<a href="https://colab.research.google.com/github/BillWorstell/Akan/blob/main/Ananse1/Ananse1Streamlit0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can create a simple interactive GUI in Python using Streamlit, a lightweight library that's perfect for deploying applications quickly, including on Google Colab. Streamlit will allow you to integrate text, images, and interact with GPT-4 API, all from a single interface with a friendly user experience.

Here’s how you can set up this interactive folklore exploration tool in Streamlit. The following code includes a GUI where users can explore the dilemmas, see paired stories, and interact with a GPT-4 API for deeper analysis or explanation of each tale. We’ll also add options for adaptive user control and seamless integration with GitHub for version tracking.

###Steps to Create and Deploy the Demo App
* Install Streamlit and other required libraries.
* Build the Streamlit GUI with options for choosing dilemmas and displaying the story pairs.
* Integrate GPT-4 API for analysis and interpretation.
* Deploy the app on Google Colab and save it to GitHub

#1. Install Streamlit and Required Libraries
In Google Colab, install Streamlit, requests (for API calls), and ngrok (for hosting the Streamlit app).

In [ ]:
!pip install streamlit requests pyngrok

###2. **Set Up the Ngrok Authentication Token**
Replace "your_authtoken" with your actual ngrok token in the following cell.

In [ ]:
#!ngrok config add-authtoken "your_authtoken"
#!ngrok authtoken "your_authtoken"
!ngrok config add-authtoken "2ol9K0GNdjTzB2GrP2i2LES3WVu_EDz1dLjEE5YJ2yMjXAob"
!ngrok authtoken "2ol9K0GNdjTzB2GrP2i2LES3WVu_EDz1dLjEE5YJ2yMjXAob"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


###3. **Mount Google drive for Excel Databases**
Mount to /content/drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

###4. **Save Your Streamlit App Code in app.py**
In a new Colab cell, copy your Streamlit app code and save it as app.py:

In [ ]:
code = """
# Import required libraries
import pandas as pd

import spacy
from IPython.display import display, HTML
import streamlit as st
import requests

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Define a dictionary for styling based on POS tags
styling_rules = {
    "NOUN": {"color": "blue", "bold": True},       # Nouns
    "PRON": {"color": "purple", "italic": True},   # Pronouns
    "VERB": {"color": "green", "underline": True}, # Verbs
    "ADV": {"color": "orange"},                   # Adverbs
    "CONJ": {"color": "red", "bold": True},        # Conjunctions
    "ADP": {"color": "cyan"},                     # Prepositions
    "DET": {"color": "gray", "italic": True},     # Articles
    "PUNCT": {"color": "black"},                 # Punctuation
    "TEXT": {"color": "brown", "font_name": "Courier New"} # Direct Speech
}

# Function to apply styling
def style_sentence(sentence):
    # Process the sentence with spaCy
    doc = nlp(sentence)
    styled_words = []

    for token in doc:
        # Get styling rules based on token POS
        style = styling_rules.get(token.pos_, {"color": "black"})
        color = style.get("color", "black")
        bold = "font-weight:bold;" if style.get("bold", False) else ""
        italic = "font-style:italic;" if style.get("italic", False) else ""
        underline = "text-decoration:underline;" if style.get("underline", False) else ""
        font = f"font-family:{style.get('font_name', 'Arial')};"

        # Create styled HTML for the word
        styled_word = f"<span style='color:{color}; {bold} {italic} {underline} {font}'>{token.text}</span>"
        styled_words.append(styled_word)

    # Combine styled words into a single HTML string
    return " ".join(styled_words)

## Example sentence
#sentence = "Ananse told his wife, Ya Nsia, 'Rise, let us go and retrieve the Sky God's stories.'"
#styled_sentence = style_sentence(sentence)

## Display the styled sentence in HTML
#display(HTML(f"<div style='font-family:Arial; font-size:16px;'>{styled_sentence}</div>"))

# Function to transfer POS styling from AtoE to Akan
def transfer_style(akan_sentence, atoe_sentence):
    # Process the AtoE sentence with spaCy to get POS tags
    doc = nlp(atoe_sentence)
    atoe_words = [token.text for token in doc]
    pos_tags = [token.pos_ for token in doc]

    # Split Akan sentence into words
    akan_words = akan_sentence.split()

    # Ensure word counts match (basic alignment)
    if len(akan_words) < len(atoe_words):
      atoe_words = atoe_words[:len(akan_words)]
    elif len(akan_words) > len(atoe_words):
      akan_words = akan_words[:len(atoe_words)]

    if len(akan_words) != len(atoe_words):
      return f"<span style='color:red;'>Error: Word count mismatch between Akan and AtoE</span>"

    # Apply styles from AtoE POS tags to Akan words
    styled_words = []
    for word, pos in zip(akan_words, pos_tags):
        style = styling_rules.get(pos, {"color": "black"})  # Default to black if no rule
        color = style.get("color", "black")
        bold = "font-weight:bold;" if style.get("bold", False) else ""
        italic = "font-style:italic;" if style.get("italic", False) else ""
        underline = "text-decoration:underline;" if style.get("underline", False) else ""
        font = f"font-family:{style.get('font_name', 'Arial')};"

        # Create styled HTML for the word
        styled_word = f"<span style='color:{color}; {bold} {italic} {underline} {font}'>{word}</span>"
        styled_words.append(styled_word)

    # Combine styled words into a single HTML string
    return " ".join(styled_words)
#
####################
#
# OpenAI API Key (replace with your key)
# OPENAI_API_KEY =
#
####################
#
# Define Pointers for each column
StoryLevel = {
  "Level": ['Learned','Learned Layman','Layman','Simple Layman','Simple','Simple Preschool','Preschool','PrePreschool'],
}

# Streamlit app setup
st.title("HOW KWAKU ANANSE (THE SPIDER) GOT ASO IN MARRIAGE")

# URL of the raw Excel file in the GitHub repository
url_Story = "https://raw.githubusercontent.com/BillWorstell/Akan/main/Ananse1/Ananse1.xlsx"

# Create an ExcelFile object
story_file = pd.ExcelFile(url_Story, engine="openpyxl")

# Dictionary to store the number of rows for each worksheet
worksheet_row_counts = {}

# Loop through all sheet names
for sheet_name in story_file.sheet_names:
    # Read the current sheet into a DataFrame
    df = story_file.parse(sheet_name)
    # Get the number of rows in the current sheet
    num_rows = df.shape[0]
    # Store the row count in the dictionary
    worksheet_row_counts[sheet_name] = num_rows

# Sidebar integration
with st.sidebar:
    st.header("Story= Ananse1")

    # User selects Level options
    #st.subheader("Level Selection")
    level_choice = st.selectbox("Story Level", StoryLevel["Level"])
    #st.write("[", level_choice, "]")

    # User selects Sentence
    NSentences= worksheet_row_counts[level_choice]
    #st.write("Number of Sentences: ", NSentences)

    sentence = st.slider("Sentence Number?", 0, NSentences, 0)
    st.write("[", sentence, "]")


# Load Excel file
story_data = story_file.parse(level_choice)

# Main section split into three columns
col1, col2, col3 = st.columns(3)

# Populate Column 1
with col1:
    st.write("### Akan")
    st.write(story_data.iloc[sentence, 0])
    akan_sentence = story_data.iloc[sentence, 0]  # Akan sentence
    atoe_sentence = story_data.iloc[sentence, 2]  # Corresponding AtoE sentence
    styled_akan = transfer_style(akan_sentence, atoe_sentence)  # Transfer styles
    st.markdown(styled_akan, unsafe_allow_html=True)  # Render styled Akan sentence

# Populate Column 2
with col2:
    st.write("### AtoE")
    st.write(story_data.iloc[sentence, 2])
    # Selected sentence
    styled_sentence = style_sentence(story_data.iloc[sentence, 2])
    # Render the styled sentence in Streamlit
    st.markdown(styled_sentence, unsafe_allow_html=True)

# Populate Column 3
with col3:
    st.write("### English")
    st.write(story_data.iloc[sentence, 1])
    # Selected sentence
    styled_sentence = style_sentence(story_data.iloc[sentence, 1])
    # Render the styled sentence in Streamlit
    st.markdown(styled_sentence, unsafe_allow_html=True)

st.write("Enjoy folklore!")
"""

with open("app.py", "w") as file:
    file.write(code)

###Key Features:
1. Independent Level Selection for Both Columns:

* Each column (Column 1 and Column 2) has its own independent drop-downs in the sidebar.
* Users can select different levels of Akan, AtoE, and English for comparison.
2. Side-by-Side Display:

* Both columns display the selected levels for Akan, AtoE, and English texts.
* Useful for comparing different translation levels.
3. Sidebar Organization:

* Clearly separates settings for Column 1 and Column 2 to make selections intuitive.
###Expected Output:
* Sidebar: Allows level selection for both columns independently.
* Main Display:
** Column 1: Displays Akan, AtoE, and English texts as per Column 1 selections.
** Column 2: Displays Akan, AtoE, and English texts as per Column 2 selections.

In [ ]:
#!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py

### 4. **Run the Streamlit App and Create the Ngrok Tunnel**
Now, run the following code to start the Streamlit app and create an ngrok tunnel to access it. Make sure only one instance of Streamlit is running to avoid conflicts.

In [ ]:
import subprocess
# Start Streamlit app on port 8501
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>

In [ ]:
from pyngrok import ngrok

# Start a single ngrok tunnel to port 8501
url = ngrok.connect(8501)
print(f"Access the Streamlit app at: {url}")


Access the Streamlit app at: NgrokTunnel: "https://f610-34-80-134-134.ngrok-free.app" -> "http://localhost:8501"


###**Step 5: Confirm Streamlit is Running on Port 8501**
To verify that the Streamlit app is accessible, run:

In [ ]:
!curl -I http://localhost:8501


HTTP/1.1 200 OK
Server: TornadoServer/6.3.3
Content-Type: text/html
Date: Tue, 17 Dec 2024 22:14:20 GMT
Accept-Ranges: bytes
Etag: "613d14f73999f92269138f7dbd22485081c10e3d5c489b103666443133746140b72bb3cb2f6d1bedcbc36d27862c97075cf91a6c552e6f2ad72acdb1faf46b6d"
Last-Modified: Tue, 17 Dec 2024 22:09:56 GMT
Cache-Control: no-cache
Content-Length: 1837
Vary: Accept-Encoding



If Streamlit is running correctly, this command should show a response. If there’s an error, check if the Streamlit app has started correctly in the cell above.

###**Important Notes:**

*   Avoid running multiple cells that initiate ngrok simultaneously, as this will exceed the session limit.
*   If you still face issues, go to the ngrok dashboard to terminate any active sessions before starting a new one in Colab.


.